In [0]:
%sql
SELECT * FROM guns.bronze.stress_level_bronze

In [0]:
%sql
CREATE OR REPLACE VIEW guns.gold.bully_depression AS 
SELECT 
  bullying,
  ROUND(AVG(depression), 2) AS avergage_depression
FROM guns.bronze.stress_level_bronze
GROUP BY bullying
ORDER BY bullying

In [0]:
%sql
SELECT * FROM guns.gold.bully_depression

In [0]:
%sql
GRANT USE CATALOG ON CATALOG guns TO interns

In [0]:
%sql
SHOW GRANTS ON CATALOG guns

In [0]:
%sql
REVOKE USE CATALOG ON CATALOG guns FROM interns

In [0]:
%sql
SHOW GRANTS ON CATALOG guns

# **Dynamic View**

In [0]:
%sql
CREATE OR REPLACE VIEW guns.gold.to_interns AS 
SELECT 
  CASE 
    WHEN is_account_group_member("interns") THEN depression
    ELSE -1
  END AS depression,
  anxiety_level
FROM guns.bronze.stress_level_bronze
WHERE 
  CASE 
    WHEN is_account_group_member("supervisor") THEN TRUE
    ELSE bullying > 2
  END
ORDER BY anxiety_level DESC;

SELECT * FROM guns.gold.to_interns

# **Row Filters**

In [0]:
%sql
CREATE OR REPLACE FUNCTION peak_bully(bullying INT)
RETURNS BOOLEAN
RETURN IF(is_account_group_member('account_users'), true, bullying > 3)

In [0]:
%sql
CREATE OR REPLACE TABLE guns.gold.full_data AS 
SELECT
  *
FROM guns.bronze.stress_level_bronze;

ALTER TABLE guns.gold.full_data SET ROW FILTER peak_bully ON (bullying);

SELECT * FROM guns.gold.full_data;

# **Column Masking**

In [0]:
%sql
CREATE OR REPLACE FUNCTION hide_bully(bullying INT)
RETURN 
  CASE 
    WHEN is_account_group_member("account_users") THEN -999
    ELSE bullying
  END;

ALTER TABLE guns.gold.full_data 
ALTER COLUMN bullying
SET MASK hide_bully;

SELECT * FROM guns.gold.full_data;